In [1]:
import sys
import numpy as np
import pandas.io.sql
import pyodbc
import pandas as pd

#from IPython.core.debugger import set_trace

In [2]:
from dmbtools.wellbore.survey import position_log
from dmbtools.wellbore.horizontal_stats_pak import h_stats, h_stats_meta
from dmbtools.geodetic.projection import GridConvergence
from dmbtools.geodetic import convert as cvt
from dmbtools.geodetic import projection as prj
from dmbtools.math import utils as mutils

#### Setup our connection to the GGX database...

In [3]:
cnxn_str = 'DRIVER={0};HOST={1};DATABASE={2};DBF={3};UID={4};PWD={5}'.format('SQL Anywhere 17', 'GGX07-MID-PRD.conchoresources.com:2638', 'GGX007_DBBU-GGX_Proj', 'E:\GGX_Proj\GGX007_DBBU\gxdb.db', 'dba', 'sql')
cnxn_str

'DRIVER=SQL Anywhere 17;HOST=GGX07-MID-PRD.conchoresources.com:2638;DATABASE=GGX007_DBBU-GGX_Proj;DBF=E:\\GGX_Proj\\GGX007_DBBU\\gxdb.db;UID=dba;PWD=sql'

In [4]:
cnxn = pyodbc.connect(cnxn_str)

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [5]:
#sorted(list(pd.read_sql("select table_name from sys.systab where table_type_str = 'BASE'", cnxn)['table_name']))

Create the list of UWI's and get the survey stations

In [6]:
#well_list_str = ','.join(["'" + str(uwi) + "'" for uwi in well_list['UWI']])
well_list_str = "'300150011700', '300150013401', '300150070300', '300150071701'" # for now these two because the project above does have any of the UWI in the Hound Dog area.
well_list_long = "'300150011700', '300150013401', '429980051251', '429991861452'" # list of wells with long laterals

In [7]:
#sql_stations = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format(well_list_str)
#sql_stations = 'select * from WELL_DIR_SRVY_STATION'
sql_stations = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format(well_list_long)
sql_wells = 'select * from WELL WHERE UWI IN ({0})'.format(well_list_long)

In [8]:
sql_wells

"select * from WELL WHERE UWI IN ('300150011700', '300150013401', '429980051251', '429991861452')"

In [9]:
well_data = pandas.read_sql(sql_wells, cnxn)

NameError: name 'cnxn' is not defined

In [ ]:
well_data.head()

In [ ]:
stations_data = pandas.read_sql(sql_stations, cnxn)

In [ ]:
stations_data.head()

In [ ]:
max_mds = stations_data.groupby(['UWI'])['STATION_MD'].max()

In [ ]:
max_mds

In [ ]:
stations_data['UWI'].drop_duplicates()

In [ ]:
list(stations_data.columns)

We sort just to be sure, but probably don't need t0...

In [ ]:
offsets = stations_data[['UWI', 'STATION_MD', 'Y_OFFSET', 'X_OFFSET', 'STATION_TVD']].sort_values(['UWI', 'STATION_MD'])

In [ ]:
offsets.head(450)

In [ ]:
tie_in_ids = offsets.groupby(['UWI'])['STATION_MD'].min()

In [ ]:
tie_in_ids

In [ ]:
tie_ins = offsets.merge(tie_in_ids, on=['UWI', 'STATION_MD'])

In [ ]:
tie_ins.head(450)

In [ ]:
readings = stations_data[['UWI', 'STATION_MD', 'INCLINATION', 'AZIMUTH']]

In [ ]:
readings[readings['UWI'] == '300150011700'].head(-5)

stats = [uwi, nlp_md]
    stats = np.append(stats, np.array(end_pts))
    stats = np.append(stats, best_fit_direction)
    stats = np.append(stats, [mean_inc, mean_az, mean_length])
    stats = np.append(stats, [lateral_length, tortuosity])
    stats = np.append(stats, [sinu, undu])

In [ ]:
h_stats_names = h_stats_meta()['names']
empty_h_stats_rec = h_stats_meta()['empty_rec']

In [ ]:
uwis = tie_ins[['UWI']]

In [ ]:
#%%debug

stats = []
#pdb.set_trace()
for uwi in tie_ins['UWI']:
    survey = readings[readings['UWI'] == uwi][['STATION_MD', 'INCLINATION', 'AZIMUTH']].to_numpy()
    tie_in = tie_ins[tie_ins['UWI'] == uwi][['Y_OFFSET', 'X_OFFSET', 'STATION_TVD']].to_numpy()[0]
    try:
        print(uwi)
        #breakpoint()
        hs = h_stats(uwi, survey, tie_in, offset=None, nominal_step=5.0, min_inc=40.0, max_inc=87.0, critical_k=1.0/15000.0, discretize_step=1.0, mean_div=3)
        stats_dict = dict(zip(h_stats_names, hs))
        stats_dict['error'] = ''
        stats.append(stats_dict)
    except Exception as ex:
        stats_dict = dict(zip(h_stats_names, empty_h_stats_rec))
        stats_dict['uwi'] = uwi
        stats_dict['error'] = str(ex)
        stats.append(stats_dict)

In [ ]:
stats

In [ ]:
stats_pd = pd.DataFrame(stats)

In [ ]:
stats_pd.head()

In [ ]:
theta = np.radians(90)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c, -s), (s, c)))
print(R)

In [ ]:
mutils.normalized([c, s])

In [ ]:
np.linalg.norm([c, s])

In [ ]:
R.shape

In [ ]:
x = np.array([[1,0,1],[1,0,1],[0,1,1],[0,1,1],[1,0,1],[1,0,1],[0,1,1],[0,1,1]])

In [ ]:
x

In [ ]:
x.shape

In [ ]:
R.dot(x[2,:2])

In [ ]:
xr = R.dot(x[:,:2].T).T

In [ ]:
xr

In [ ]:
y = np.array(x, copy=True)

In [ ]:
y[:,:2] = xr

In [ ]:
y

In [ ]:
ggx_project_path = '\\GGX07-MID-PRD\Discovery-3bb3ef81-f9ed-4c1f-8e7b-5a2dab4ca8e2$\GGX007_DBBU'
oneuwi = '429991861452'
epgs_geo_27 = 4267
epsg_crs_27 = 32039
spcs_27 = 'SPCS27 - Texas Central'
epgs_geo_83 = 4269
epsg_crs_83 = 32139
spcs_27 = 'SPCS83 - Texas Central'
latlon = [31.08137, -103.46939]

In [ ]:
# 31 43 21.85539(N) 103 49 14.83427(W)
#                     North         East     Units Scale Factor Converg.
# SPC TX C     - 3,233,113.731   369,538.808   MT  0.99996081   -1 47 46.5
lat_CC0554 = cvt.dms2decdeg(31, 43, 21.85539)
lon_CC0554 = cvt.dms2decdeg(-103, 49, 14.83427)
con_CC0554 = cvt.dms2decdeg(-1, 47, 46.5)

In [ ]:
gc = GridConvergence(4269, 32139)

In [ ]:
gc._prj._from_proj

In [ ]:
gc._prj._to_proj

In [ ]:
gc.calc(np.array([[lat_CC0554, lon_CC0554]]))

In [ ]:
con_CC0554

In [ ]:
cvt.decdeg2dms(gc.calc(np.array([[lat_CC0554, lon_CC0554]])))

In [ ]:
sql_stat_one = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format("'429980051251'")
one_stat_dat = pandas.read_sql(sql_stat_one, cnxn)[['Y_OFFSET', 'X_OFFSET', 'STATION_TVD']].to_numpy()
one_stat_dat

In [ ]:
one_stat_dat.shape

In [ ]:
sql_well_one = 'select * from WELL WHERE UWI IN ({0})'.format("'429980051251'")
one_well_dat = pandas.read_sql(sql_well_one, cnxn)[['SURFACE_LATITUDE', 'SURFACE_LONGITUDE']].to_numpy()
one_well_dat

In [ ]:
one_stat_nev = prj.toTrueNorthNEV(one_stat_dat, one_well_dat[0,0], one_well_dat[0,1], epgs_geo_27, epsg_crs_27, forward=False)

In [ ]:
one_stat_nev

In [ ]:
cvt.usft2meters(one_stat_nev)

In [ ]:
one_stat_ll = cvt.lmp(cvt.usft2meters(one_stat_nev), one_well_dat[0,0], one_well_dat[0,1], datum='nad27')
one_stat_ll

In [ ]:
sql_ll_one = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format("'429980051251'")
one_ll_ggx = pandas.read_sql(sql_ll_one, cnxn)[['GX_STATION_LATITUDE', 'GX_STATION_LONGITUDE']].to_numpy()
one_ll_ggx

In [ ]:
one_stat_ll - one_ll_ggx

In [ ]:
(one_stat_ll - one_ll_ggx).max()

In [ ]:
0.0007502965498424885

In [ ]:
cvt.decdeg2dms((one_stat_ll - one_ll_ggx).max())

In [ ]:
(0.0, 0.0, 2.7010675794329586)

In [ ]:
tvds_geo = np.vstack([one_stat_ll[-1,:], one_ll_ggx[-1:]])
tvds_geo

In [ ]:
tvds_geo.shape

In [ ]:
tvds_crt = prj.Project(epgs_geo_27, epsg_crs_27).forward(tvds_geo)

In [ ]:
tvds_crt

In [ ]:
tvds_crt[0]

In [ ]:
np.linalg.norm(tvds_crt[0] - tvds_crt[1])

In [ ]:
350.2986141670411

In [ ]:
ggx_lat_crs = prj.Project(epgs_geo_27, epsg_crs_27).forward(np.array([[one_well_dat[0,0], one_well_dat[0,1]]]))

In [ ]:
ggx_lat_crs

In [ ]:
one_stat_dat[-1,:1]

In [ ]:
ggx_lat_crs[0] + one_stat_dat[-1,:1]

In [ ]:
prj.Project(epgs_geo_27, epsg_crs_27).inverse([ggx_lat_crs[0] + one_stat_dat[-1,:1]])

In [ ]:
one_ll_ggx[-1]